In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from skimage.exposure import rescale_intensity
from skimage.transform import resize

files = {
    'F140M': 'jw02739-o011_t002_nircam_clear-f140m_i2d.fits',
    'F210M': 'jw02739-o011_t002_nircam_clear-f210m_i2d.fits',
    'F300M': 'jw02739-o011_t002_nircam_clear-f300m_i2d.fits',
    'F460M': 'jw02739-o011_t002_nircam_clear-f460m_i2d.fits',
}

# Load and preprocess each image
def load_and_clean(filename):
    data = fits.open(filename)[1].data
    data = np.nan_to_num(data, nan=0.0)
    data[data < 0] = 0
    return np.log10(data + 1e-3)

def normalize(image):
    return rescale_intensity(image, in_range='image', out_range=(0, 1))

# Load all four images
f140 = normalize(load_and_clean(files['F140M']))
f210 = normalize(load_and_clean(files['F210M']))
f300 = normalize(load_and_clean(files['F300M']))
f460 = normalize(load_and_clean(files['F460M']))

# Find the smallest shape among all images
min_shape = (min(f140.shape[0], f210.shape[0], f300.shape[0], f460.shape[0]),
             min(f140.shape[1], f210.shape[1], f300.shape[1], f460.shape[1]))

# Resize all images to the smallest shape
f140_resized = resize(f140, min_shape, mode='reflect', anti_aliasing=True)
f210_resized = resize(f210, min_shape, mode='reflect', anti_aliasing=True)
f300_resized = resize(f300, min_shape, mode='reflect', anti_aliasing=True)
f460_resized = resize(f460, min_shape, mode='reflect', anti_aliasing=True)

# Build RGB channels
r = f300_resized + f460_resized * 1.0
g = f210_resized + f300_resized + f460_resized * 0.5
b = f140_resized + f210_resized

# Stack and normalize RGB image
rgb = np.stack([r, g, b], axis=-1)
rgb = np.clip(rgb / np.max(rgb), 0, 1)

plt.figure(figsize=(12, 12))
plt.imshow(rgb, origin='lower')
plt.title("JWST Color Composite: F140M (Blue), F210M (Cyan), F300M (Yellow), F460M (Orange)", fontsize=14)
plt.axis('off')
plt.show()
